# Dyson Protocol Scripting Guide

This guide provides an end-to-end demonstration of the Dyson Protocol Script Module for developers. It covers script management, execution, data handling, and web access through name resolution in the least number of steps.

## Fetch Your Address

First, we'll retrieve the address associated with the 'alice' account. This address will serve as our identity throughout this guide and will be referenced in subsequent commands.


In [1]:
[address] = ! dysond keys show alice -a
print(address)

dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz


## Update Script on Chain
Now, let's deploy our script to the blockchain. We'll create a simple Python script with two functions:
1. An `add` function that performs basic arithmetic
2. A WSGI application that serves a welcome HTML page when accessed via web

This demonstrates how Dyson Protocol enables both computational functions and web hosting capabilities.


In [2]:
import os

code = """
def add(a, b):
    print(f"Adding {a} and {b}")
    return {"a": a, "b": b, "add_result": a + b}

def wsgi(environ, start_response):
    status = '200 OK'
    headers = [('Content-type', 'text/html')]
    start_response(status, headers)
    return [b'''
<html>
    <body>
        <h1>Hello from Dyson Protocol!</h1>
    </body>
</html>''']
"""
import tempfile
import json

with tempfile.NamedTemporaryFile(suffix='.py', delete=True) as tmp:
    tmp.write(code.encode())
    tmp.flush()
    path = tmp.name
    
    tx = ! dysond tx script update --code-path $path \
        --from alice \
        -y | dysond query wait-tx -o json
    
json.loads('\n'.join(tx))

{'height': '76824',
 'txhash': '86EAE6F6DD92B3F871FD48A0146BCFE4540BC04AB61DB0BCE52BAC7CFF5EC4C7',
 'codespace': '',
 'code': 0,
 'data': '12360A302F6479736F6E70726F746F636F6C2E7363726970742E76312E4D7367557064617465536372697074526573706F6E736512020802',
 'raw_log': '',
 'logs': [],
 'info': '',
 'gas_wanted': '200000',
 'gas_used': '55966',
 'tx': None,
 'timestamp': '',
 'events': [{'type': 'tx',
   'attributes': [{'key': 'acc_seq',
     'value': 'dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz/90',
     'index': True}]},
  {'type': 'tx',
   'attributes': [{'key': 'signature',
     'value': 'KMZWvnMfbXhJY8xAPckO8lEwJVe/LoTAmL60DJHJwJo6Pw2Zs5FRFXp2av9DD/Lv8RlY7FzeEZB3V1tOJgI9Fg==',
     'index': True}]},
  {'type': 'message',
   'attributes': [{'key': 'action',
     'value': '/dysonprotocol.script.v1.MsgUpdateScript',
     'index': True},
    {'key': 'sender',
     'value': 'dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz',
     'index': True},
    {'key': 'module', 'value': 'script', 'index': T

## Access Script via Web Interface
Dyson Protocol allows scripts to be accessed as web applications through the WSGI interface. Let's access our script directly using its address. This demonstrates how Dyson Protocol enables decentralized web hosting without traditional servers.

We'll use the script address to construct a URL that points to our deployed application. The format is:
`http://<script_address>.example.com`

For local development, we'll use localhost:8000 as our domain suffix.

In [3]:
[output] = ! dysond config get app api.address
port = output.split(":")[-1].strip("\"")

dwapp_url = f"http://{address}.localhost:{port}"

print(f"Accessing your DWapp at '{dwapp_url}'")
output = ! curl -s "$dwapp_url/hi" -v
output = "\n".join(output).strip()
print(output)
assert "Hello from Dyson Protocol!" in output, "Expected 'Hello from Dyson Protocol!' in output, got: " + output

Accessing your DWapp at 'http://dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz.localhost:2417'


* Host dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz.localhost:2417 was resolved.
* IPv6: ::1
* IPv4: 127.0.0.1
*   Trying [::1]:2417...
* connect to ::1 port 2417 from ::1 port 58830 failed: Connection refused
*   Trying 127.0.0.1:2417...
* Connected to dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz.localhost (127.0.0.1) port 2417
> GET /hi HTTP/1.1
> Host: dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz.localhost:2417
> User-Agent: curl/8.6.0
> Accept: */*
> 
< HTTP/1.1 200 OK
< Content-Length: 82
< Content-Type: text/html
< Date: Wed, 11 Jun 2025 18:08:55 GMT
< Server: WSGIServer/0.2 CPython/3.11.4
< X-Server-Time: 1749665335
< 
{ [82 bytes data]
* Connection #0 to host dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz.localhost left intact

<html>
    <body>
        <h1>Hello from Dyson Protocol!</h1>
    </body>
</html>


## Query Script Information
Let's examine the script we just deployed to the blockchain. This query retrieves the script's metadata and code content, allowing us to verify our update was successful.


In [4]:
import json

output = ! dysond query script script-info --address "$address" -o json 

print("\n".join(output))
script_info = json.loads('\n'.join(output))
print(f"✓ Script query successful for address: {address}")


{
  "script": {
    "address": "dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz",
    "version": "2",
    "code": "def add(a, b):\n    print(f\"Adding {a} and {b}\")\n    return {\"a\": a, \"b\": b, \"add_result\": a + b}\n\n\ndef wsgi(environ, start_response):\n    status = \"200 OK\"\n    headers = [(\"Content-type\", \"text/html\")]\n    start_response(status, headers)\n    return [\n        b\"\"\"\n\u003chtml\u003e\n    \u003cbody\u003e\n        \u003ch1\u003eHello from Dyson Protocol!\u003c/h1\u003e\n    \u003c/body\u003e\n\u003c/html\u003e\"\"\"\n    ]\n\n"
  }
}
✓ Script query successful for address: dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz


## Execute Script
## Execute Script Function
Now we'll invoke the `add` function we deployed in our script. This demonstrates how Dyson Protocol enables 
decentralized computation by executing functions directly on the blockchain. We'll pass the arguments `5` and `7`, 
and observe how the function processes these values and returns the calculated sum of `12` along with additional metadata.

In [5]:
! dysond tx script exec \
    --script-address "$address" \
    --function-name add \
    --args '[5, 7]' \
    --from alice \
    -y \
    -o json  | dysond query wait-tx -o json | python ../scripts/parse_exec_script_tx.py

{
  "code": 0,
  "script_result": {
    "result": {
      "cumsize": 2041,
      "exception": null,
      "gas_limit": 200000,
      "nodes_called": 28,
      "result": {
        "a": 5,
        "add_result": 12,
        "b": 7
      },
      "script_gas_consumed": 32566,
      "stdout": "Adding 5 and 7\n"
    },
    "attached_message_results": []
  },
  "raw_log": "",
  "events": [
    {
      "type": "tx",
      "attributes": [
        {
          "key": "acc_seq",
          "value": "dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz/91",
          "index": true
        }
      ]
    },
    {
      "type": "tx",
      "attributes": [
        {
          "key": "signature",
          "value": "FpCdbx36pzqCuCB2EO8sjkfbUQ9Vdb2uB9dHhwlisKZpcmfoN+aL6eCoDpKu4U8ok2YvBmAJl/2NlReApQJlzw==",
          "index": true
        }
      ]
    },
    {
      "type": "message",
      "attributes": [
        {
          "key": "action",
          "value": "/dysonprotocol.script.v1.MsgExec",
          "index":

# Encoding JSON for Blockchain Operations
Your project may require converting complex JSON structures into a compact binary format for efficient on-chain storage and transmission. The following example demonstrates how to encode a standard transaction message into its binary representation.

In [6]:
! dysond query script encode-json --json '{\
  "@type": "/cosmos.bank.v1beta1.MsgSend", \
  "from_address": "dys1example1", \
  "to_address": "dys1example2", \
  "amount": [ { "denom": "dys", "amount": "100" } ] \
}' -o json

{
  "bytes": "CgxkeXMxZXhhbXBsZTESDGR5czFleGFtcGxlMhoKCgNkeXMSAzEwMA=="
}


## Decode Bytes
Decoding Binary Data
In this step, we'll convert the previously encoded binary data back into its original JSON format. This bidirectional conversion capability is essential for working with blockchain data that needs to be both efficiently stored on-chain and human-readable when retrieved.

In [7]:
! dysond query script decode-bytes --bytes "CgxkeXMxZXhhbXBsZTESDGR5czFleGFtcGxlMhoKCgNkeXMSAzEwMA=="  --type-url "/cosmos.bank.v1beta1.MsgSend" -o json | jq

{
  "json": "{\"@type\":\"/cosmos.bank.v1beta1.MsgSend\",\"from_address\":\"dys1example1\",\"to_address\":\"dys1example2\",\"amount\":[{\"denom\":\"dys\",\"amount\":\"100\"}]}"
}


## Commit Name Registration
More details on name registration can be found in the Name Service section of the documentation.
Commit to registering a name using a computed hash. First, compute the hash.

In [8]:
import random
import string

def random_string(length=10):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

name = f"alice-{random_string(5)}.dys"
salt = random_string(10)

[hex_hash] = ! dysond query nameservice compute-hash \
    --name "$name" \
    --salt "$salt" \
    --committer "$address" \
    -o json | jq '.hex_hash' -r

print(f"Name: {name}")
print(f"Salt: {salt}")
print(f"Hex Hash: {hex_hash}")

Name: alice-k89a6.dys
Salt: e4kdzy6r08
Hex Hash: 25cada1530e94bb3af19df4981dd8789f730325762d8162737adb3c5552fd9f6


In [9]:
valuation = '100dys'
! dysond tx nameservice commit --commitment "$hex_hash" --valuation "$valuation" --from alice -y | dysond query wait-tx -o json | jq -M

{
  "height": "76837",
  "txhash": "4A835E154D76069489BE2EC2F5A594596E7E27003EF3E82EEE41B851982ACC4C",
  "codespace": "",
  "code": 0,
  "data": "12310A2F2F6479736F6E70726F746F636F6C2E6E616D65736572766963652E76312E4D7367436F6D6D6974526573706F6E7365",
  "raw_log": "",
  "logs": [],
  "info": "",
  "gas_wanted": "200000",
  "gas_used": "39792",
  "tx": null,
  "timestamp": "",
  "events": [
    {
      "type": "tx",
      "attributes": [
        {
          "key": "acc_seq",
          "value": "dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz/92",
          "index": true
        }
      ]
    },
    {
      "type": "tx",
      "attributes": [
        {
          "key": "signature",
          "value": "OGXQwDXhuTa1qDS0QHzHkjn/tyQywzvKs4CWdGag6Y47AESQn4NvjLFbFPV3K6HXsA+mQ0ijC5KA3G336v8lRQ==",
          "index": true
        }
      ]
    },
    {
      "type": "message",
      "attributes": [
        {
          "key": "action",
          "value": "/dysonprotocol.nameservice.v1.MsgCommit",
     

## Reveal Name Registration
Reveal the name to complete registration.

In [10]:
! dysond tx nameservice reveal \
    --name "$name" \
    --salt "$salt" \
    --from alice \
    -y | dysond query wait-tx -o json | jq -M

{
  "height": "76840",
  "txhash": "D3BA68033544552A5265E41BD33323E1091A00E0FEBD708119760F701EE1B99A",
  "codespace": "",
  "code": 0,
  "data": "12310A2F2F6479736F6E70726F746F636F6C2E6E616D65736572766963652E76312E4D736752657665616C526573706F6E7365",
  "raw_log": "",
  "logs": [],
  "info": "",
  "gas_wanted": "200000",
  "gas_used": "120660",
  "tx": null,
  "timestamp": "",
  "events": [
    {
      "type": "tx",
      "attributes": [
        {
          "key": "acc_seq",
          "value": "dys1tvhkv3gqr90jpycaky02xa5ukhaxllu38wawhz/93",
          "index": true
        }
      ]
    },
    {
      "type": "tx",
      "attributes": [
        {
          "key": "signature",
          "value": "RLibZwdBpqcYSLwpk15TFEpn2bJIayPVG4wg71OB+4pTXmfUvIExJl4DtpPCzRsFq0aTqPvS2Gqh1XEQdB5YSg==",
          "index": true
        }
      ]
    },
    {
      "type": "message",
      "attributes": [
        {
          "key": "action",
          "value": "/dysonprotocol.nameservice.v1.MsgReveal",
    

## Set Destination for Name
Set the destination of the registered name to Alice's address.

In [11]:
! dysond tx nameservice set-destination \
    --name "$name" \
    --destination "$address" \
    --from alice \
    -y \
    -o json | jq -M

{
  "height": "0",
  "txhash": "BED5CEFA7BE2CA2706BC864308C50C93F2D01EF04C82902B272809543B973CAF",
  "codespace": "",
  "code": 0,
  "data": "",
  "raw_log": "",
  "logs": [],
  "info": "",
  "gas_wanted": "0",
  "gas_used": "0",
  "tx": null,
  "timestamp": "",
  "events": []
}


## Access Script via Name
Access the script via the registered name to demonstrate decentralized web hosting.

In [12]:
[output] = ! dysond config get app api.address
port = output.split(":")[-1].strip("\"")

dwapp_url = f"http://{name}.localhost:{port}"

print(f"Accessing your DWapp at '{dwapp_url}'")
output = ! curl -s "$dwapp_url/hi" -v
output = "\n".join(output).strip()
print(output)
assert "Hello from Dyson Protocol!" in output, "Expected 'Hello from Dyson Protocol!' in output, got: " + output

Accessing your DWapp at 'http://alice-k89a6.dys.localhost:2417'


* Host alice-k89a6.dys.localhost:2417 was resolved.
* IPv6: ::1
* IPv4: 127.0.0.1
*   Trying [::1]:2417...
* connect to ::1 port 2417 from ::1 port 61316 failed: Connection refused
*   Trying 127.0.0.1:2417...
* Connected to alice-k89a6.dys.localhost (127.0.0.1) port 2417
> GET /hi HTTP/1.1
> Host: alice-k89a6.dys.localhost:2417
> User-Agent: curl/8.6.0
> Accept: */*
> 
< HTTP/1.1 200 OK
< Content-Length: 82
< Content-Type: text/html
< Date: Wed, 11 Jun 2025 18:08:58 GMT
< Server: WSGIServer/0.2 CPython/3.11.4
< X-Server-Time: 1749665338
< 
{ [82 bytes data]
* Connection #0 to host alice-k89a6.dys.localhost left intact

<html>
    <body>
        <h1>Hello from Dyson Protocol!</h1>
    </body>
</html>
